In [ ]:
from dotenv import load_dotenv
from crewai import Agent, Task, Crew, Process
from crewai_tools import DallETool

load_dotenv()

verbose = False # A value of `True` will instruct the Crew to output information while they're in progress; or `False` to get no information and just the result.

# ───────
# AGENTS:
# ───────
producer = Agent(
    role="Travel Agent Producer",
    goal="Produce, using your Crew, content about a location",
    backstory="""You are an expert producer and servant leader, polite, skilled in gathering input from a human user and overseeing complex projects and guiding teams to success.
                 Delegating tasks to your Crew is crucial to success.
                 Sensitive information, like Personally Identifiable Information and other private information, should never be asked for, never communicated, never included.""",
    allow_delegation=True,
    verbose=verbose,
    llm="openai/gpt-4.1",
)

user_surveyor = Agent(
    role="User Surveyor",
    goal="Ask questions to the human user, and return the response",
    backstory="You are an expert communicator. You ask for input from the human user and return their response as plain text.",
    verbose=verbose,
)

researcher = Agent(
    role="Researcher",
    goal="Research information about a location",
    backstory="""You are a seasoned researcher who specialises in finding out about locations. You excel in finding out about the landmarks, activities available and things to do in a location.""",
    verbose=verbose,
    llm="openai/o3-mini",
)

weather_forecaster = Agent(
    role="Weather Forecaster",
    goal="Write a description about the weather",
    backstory="You are an expert Weather Forecaster who writes an engaging paragraph about the weather, very much like TV Weather Presenters, using well understood meteorological adjectives.",
    verbose=verbose,
    llm="openai/o3-mini",
)

artist = Agent(
    role="Promotional Artist",
    goal="Draw images that advertise and promote a location, its weather and its available activities and landmarks",
    backstory="""You are talented artist with a passion for producing fun, light-hearted, polite, friendly and inviting promotional imagery that is suitable for people of all ages.
                 You must use the DallE Tool to draw the image.""",
    tools=[DallETool()],
    verbose=verbose,
)

# ─────
# TASK:
# ─────
produce = Task(
    description="""Ask the user to input a location, either a town or city, in the United Kingdom; and then make an engaging,
                   fun and inviting image about the landmarks, activities available and things to do in that location, and the current weather in that location.
                   Ask the human user what style they want the image in.""",
    expected_output="A URL to the generated image (about the location, things to do there, and the current weather there); and a description of the image.",
    agent=artist,
    human_input=True,
)

result = Crew(
    agents=[user_surveyor, researcher, weather_forecaster, artist],
    tasks=[produce],
    process=Process.hierarchical,
    manager_agent=producer,
    versbose=verbose,
    memory=True,
    planning=True,
).kickoff()

print(result)
